In [18]:
import pandas as pd
import os
import geopandas as gpd
#import matplotlib.pyplot as plt
import contextily as ctx
import pyodbc
import pymssql
import sqlalchemy
import gzip
import csv
import json
import glob
import urllib
from sqlalchemy import create_engine
import tqdm


In [2]:
#load 2019 data
data_path = r'\\AWS-Model10\Model Data 2\INRIX'
os.chdir(data_path)

trips_header = pd.read_csv(os.path.join(r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1', 'schema', 'TripBulkReportTripsHeaders.csv'))
trips_df = pd.read_csv(os.path.join(r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trips\trips.csv.gz'), 
                       compression='gzip', names = trips_header.columns)
                       

traj_header = pd.read_csv(os.path.join(r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1', 'schema', 'TripBulkReportTrajectoriesHeaders.csv'))
#traj_df = pd.read_csv(os.path.join(r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trajstrajs.csv.gz'), 
#                       compression='gzip', names = trips_header.columns)

In [ ]:
traj_header = pd.read_csv(os.path.join(r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1', 'schema', 'TripBulkReportTrajectoriesHeaders.csv'))

In [ ]:
traj_df = pd.read_csv(os.path.join(r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trajs\trajs.csv.gz'), 
                       compression='gzip', names = traj_header.columns)

In [5]:
traj_df

,TripId,DeviceId,ProviderId,TripTimezone,TrajIdx,TrajRawDistanceM,TrajRawDurationMillis,SegmentId,SegmentIdx,LengthM,CrossingStartOffsetM,CrossingEndOffsetM,CrossingStartDateUtc,CrossingEndDateUtc,CrossingSpeedKph,OnRoadNetworkSnapCount,ErrorCodes
0,5551c27c3fb087dbede43779bc231bbd,912167cea504542b41d4549efad363db,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,20425.316163,2408949,-687694073_1,150,42.356,0.0,42.356,2019-05-06T14:03:41.032Z,2019-05-06T14:03:59.330Z,8.333045,0,NaN
1,35ff6cd95babdba06e2e42be3c3b842c,eca22f65417e3eca4befc1c00aa30243,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,16674.167125,1336016,594056009_1,17,206.250,0.0,206.250,2019-05-06T18:42:29.753Z,2019-05-06T18:42:49.730Z,37.167119,0,NaN
2,0ee60c7a83c9bfbbe5e8ff4592380288,80f828fb533056faee4cd17c2b6f383d,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,3745.049614,678018,61014862_0,38,63.760,0.0,63.760,2019-05-05T02:47:57.803Z,2019-05-05T02:48:09.014Z,20.472955,0,NaN
3,2a81b3a61cb207a728a16dc8d7741ad4,2b43f63905927faa56b523243c677486,f033ab37c30201f73f142449d037028d,America/Los_Angeles,0,32005.077191,1865946,292080442_0,85,58.177,0.0,58.177,2019-05-05T05:46:41.774Z,2019-05-05T05:46:44.294Z,83.097449,0,NaN
4,ab0f764110826aa970f657e39b908c16,84b57b6ef9437395ce5cce9e086cfaa4,f033ab37c30201f73f142449d037028d,America/Los_Angeles,0,13393.828158,1668048,250022448_8,151,83.854,0.0,83.854,2019-05-06T21:10:10.085Z,2019-05-06T21:10:30.585Z,14.725469,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2836658,df0ee46082df4c7e68ed21f6e98a9f25,801572fb2a8e2f08cc7173aac3ef5e16,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,12424.866263,1056977,444494470_7,55,58.607,0.0,58.607,2019-05-09T22:22:51.111Z,2019-05-09T22:22:55.454Z,48.575220,0,NaN
2836659,2f4433f483990f76d8fb60d850ce86ef,47147ab5a8de355b53d3742815dbd8df,45c48cce2e2d7fbdea1afc51c7c6ad26,America/Los_Angeles,0,160039.170117,6996977,6503478_1,209,179.811,0.0,179.811,2019-05-07T11:36:26.767Z,2019-05-07T11:36:32.994Z,103.961540,0,NaN
2836660,b8b1a1cee4bff2aa4cd6130c5a09bba5,c5427319c32960efb432923bcbc56042,f033ab37c30201f73f142449d037028d,America/Los_Angeles,0,81902.836401,4636016,451985518_0,134,76.270,0.0,76.270,2019-05-08T22:57:57.727Z,2019-05-08T22:58:01.371Z,75.351329,0,NaN
2836661,65ea58a34bc313e9c9ba3c4dff0796a9,c822735f11f12911d4d9f91aea82f682,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,12500.912355,1950030,428045786_11,170,136.927,0.0,136.927,2019-05-08T01:29:41.028Z,2019-05-08T01:29:48.810Z,63.344965,0,NaN


In [7]:
traj_df.groupby('TripId')['TripId'].count().reset_index(name='count').sort_values(by=['count'])

,TripId,count
399251,804f938945f5e4b58e14d422610877fd,1
378494,79add776fc437503f0a0008429e1e37b,1
378492,79ad999a71641e8a5fc5a7bd274c4f01,1
378491,79ad9558d65d9c8a14ffe8cbca834790,1
378490,79ad84ecbb9705fda2c9fb8da844258d,1
...,...,...
435235,8bcbc069e2c15518099615d6f9d91ac8,76
337217,6c78dd33e1fbf3da2e0eb83f3d328d17,81
276166,58db5cac146fdd4d2d0e1efe0b5a0591,93
101470,20c6f3ff46419c92f4c5365708e973ac,100


Separating Medium trucks from Heavy trucks

Upload trip table

In [ ]:
trips_header = pd.read_csv(os.path.join(r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1', 'schema', 'TripBulkReportTripsHeaders.csv'))
trips_df = pd.read_csv(os.path.join(r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trips\trips.csv.gz'), 
                       compression='gzip', names = trips_header.columns)

In [22]:
list(traj_with_trips.columns)

['TripId',
 'DeviceId',
 'ProviderId',
 'TripTimezone',
 'TrajIdx',
 'TrajRawDistanceM',
 'TrajRawDurationMillis',
 'SegmentId',
 'SegmentIdx',
 'LengthM',
 'CrossingStartOffsetM',
 'CrossingEndOffsetM',
 'CrossingStartDateUtc',
 'CrossingEndDateUtc',
 'CrossingSpeedKph',
 'OnRoadNetworkSnapCount',
 'ErrorCodes',
 'VehicleWeightClass']

In [4]:
#choosing columns that we need
cols = ['TripId','VehicleWeightClass']
trips_df_upd = trips_df[cols]

In [18]:
#left join the traj table
traj_with_trips = pd.merge(traj_df, trips_df_upd, how="left", on=["TripId"])

In [33]:
#Changing Time Zone and identofy times of week
traj_with_trips['StartDate_upd'] = (pd.to_datetime(traj_with_trips['CrossingStartDateUtc'])
                     .dt.tz_convert('America/Los_Angeles')) 
traj_with_trips["StartDOW"] = traj_with_trips["StartDate_upd"].apply(lambda x: x.strftime('%A'))
traj_with_trips["StartDate_dmy"] = traj_with_trips["StartDate_upd"].apply(lambda x: x.strftime('%d%m%Y'))
traj_with_trips["StartTime_h"] = traj_with_trips["StartDate_upd"].apply(lambda x: x.strftime('%H'))

In [34]:
#Identify weekends and delete them
weekdays = ["Monday", "Tuesday","Wednesday","Thursday","Friday"]
trips_df_weekdays = traj_with_trips[traj_with_trips.StartDOW.isin(weekdays)]


In [53]:
traj_with_trips_medium.head()

,TripId,DeviceId,ProviderId,TripTimezone,TrajIdx,TrajRawDistanceM,TrajRawDurationMillis,SegmentId,SegmentIdx,LengthM,...,CrossingStartDateUtc,CrossingEndDateUtc,CrossingSpeedKph,OnRoadNetworkSnapCount,ErrorCodes,VehicleWeightClass,StartDate_upd,StartDOW,StartDate_dmy,StartTime_h
0,5551c27c3fb087dbede43779bc231bbd,912167cea504542b41d4549efad363db,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,20425.316163,2408949,-687694073_1,150,42.356,...,2019-05-06T14:03:41.032Z,2019-05-06T14:03:59.330Z,8.333045,0,NaN,2,2019-05-06 07:03:41.032000-07:00,Monday,06052019,07
1,35ff6cd95babdba06e2e42be3c3b842c,eca22f65417e3eca4befc1c00aa30243,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,16674.167125,1336016,594056009_1,17,206.250,...,2019-05-06T18:42:29.753Z,2019-05-06T18:42:49.730Z,37.167119,0,NaN,2,2019-05-06 11:42:29.753000-07:00,Monday,06052019,11
4,ab0f764110826aa970f657e39b908c16,84b57b6ef9437395ce5cce9e086cfaa4,f033ab37c30201f73f142449d037028d,America/Los_Angeles,0,13393.828158,1668048,250022448_8,151,83.854,...,2019-05-06T21:10:10.085Z,2019-05-06T21:10:30.585Z,14.725469,0,NaN,2,2019-05-06 14:10:10.085000-07:00,Monday,06052019,14
6,c0d53f956c8aa7fe36f856a4cb6e98f2,42072bcef470c69466ea9a230bad2950,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,2190.888759,1036016,736546441_8,9,8.079,...,2019-05-06T21:55:12.091Z,2019-05-06T21:55:12.837Z,39.014110,0,NaN,2,2019-05-06 14:55:12.091000-07:00,Monday,06052019,14
8,0b30b6b5f3ee8a81d0a037fe7196c18e,1f7ba5b6271aa512b60b2b68de7815b9,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,66480.166878,5725966,777915505_3,391,62.846,...,2019-05-03T22:58:17.484Z,2019-05-03T22:58:20.710Z,70.132538,0,NaN,2,2019-05-03 15:58:17.484000-07:00,Friday,03052019,15


In [35]:
#separating medium from heavy trucks
traj_with_trips_heavy = trips_df_weekdays[trips_df_weekdays['VehicleWeightClass']==3]
traj_with_trips_medium = trips_df_weekdays[trips_df_weekdays['VehicleWeightClass']==2]

In [55]:
traj_with_trips_medium.groupby(['SegmentId','StartDate_dmy','StartTime_h']).size().reset_index(name='counts').sort_values(by=['counts'])

,SegmentId,StartDate_dmy,StartTime_h,counts
0,-100026065_0,14052019,16,1
1238564,427901478_4,16052019,08,1
1238563,427901478_4,15052019,16,1
1238562,427901478_4,15052019,15,1
1238561,427901478_4,15052019,14,1
...,...,...,...,...
1184693,376294459_0,08052019,14,8
705059,124045488_1,06052019,08,8
705060,124045488_1,06052019,09,8
986816,305335210_0,06052019,12,8


In [19]:
temp = traj_with_trips_medium.groupby(['SegmentId','StartDate_dmy','StartTime_h']).size().reset_index(name='counts')
temp.reset_index(inplace = True)
rename(columns={"SegmentId": "trips", "index": "SegmentId"})


NameError: name 'rename' is not defined

In [69]:
temp

,index,SegmentId,StartDate_dmy,StartTime_h,counts
0,0,-100026065_0,14052019,16,1
1,1,-100095907_2,23052019,20,1
2,2,-100095907_4,28052019,22,1
3,3,-100163147_0,03052019,12,1
4,4,-100163147_0,06052019,08,1
...,...,...,...,...,...
1883237,1883237,99964390_2,02052019,07,1
1883238,1883238,99979708_0,13052019,16,1
1883239,1883239,99979708_0,15052019,10,1
1883240,1883240,99979708_4,02052019,05,1


In [72]:
temp = temp.rename(columns={"SegmentId": "trips", "index": "SegmentId"})

In [76]:
temp2 = temp[temp['trips'].str.contains('-')]

In [79]:
temp2['trips']

0         -100026065_0
1         -100095907_2
2         -100095907_4
3         -100163147_0
4         -100163147_0
              ...     
641132     -99964390_9
641133     -99979708_2
641134     -99979708_4
641135     -99979708_4
641136     -99979708_4
Name: trips, Length: 641137, dtype: object

In [32]:
pd.DataFrame(traj_with_trips_medium['SegmentId'].value_counts())

,SegmentId
124045488_1,431
124045486_0,389
376294459_0,376
451985516_0,376
376294458_0,373
...,...
-291552716_0,1
-6132383_14,1
-12189650_5,1
-6134739_11,1


In [23]:
import tqdm

In [6]:
path =r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trajs' 
File_List= glob.glob(path + "/*.gz")
i_trips_full_med = pd.DataFrame()
j_trips_full_med = pd.DataFrame()

i_trips_full_heavy = pd.DataFrame()
j_trips_full_heavy = pd.DataFrame()
list_ = []
for file_ in tqdm.tqdm(File_List):
    traj_df = pd.read_csv(file_, compression='gzip',names = traj_header.columns,sep=',',  error_bad_lines=False)
    print(file_)
    
    #left join the traj table
    traj_with_trips = pd.merge(traj_df, trips_df_upd, how="left", on=["TripId"])
    del traj_df
    
    #Changing Time Zone and identofy times of week
    traj_with_trips['StartDate_upd'] = (pd.to_datetime(traj_with_trips['CrossingStartDateUtc'])
                         .dt.tz_convert('America/Los_Angeles')) 
    traj_with_trips["StartDOW"] = traj_with_trips["StartDate_upd"].apply(lambda x: x.strftime('%A'))
    traj_with_trips["StartDate_dmy"] = traj_with_trips["StartDate_upd"].apply(lambda x: x.strftime('%d%m%Y'))
    traj_with_trips["StartTime_h"] = traj_with_trips["StartDate_upd"].apply(lambda x: x.strftime('%H'))
    
    #Identify weekends and delete them
    weekdays = ["Monday", "Tuesday","Wednesday","Thursday","Friday"]
    trips_df_weekdays = traj_with_trips[traj_with_trips.StartDOW.isin(weekdays)]
    
    #separating medium from heavy trucks
    #traj_with_trips_heavy = trips_df_weekdays[trips_df_weekdays['VehicleWeightClass']==3]
    traj_with_trips_medium = trips_df_weekdays[trips_df_weekdays['VehicleWeightClass']==2]
    
    trips_summary_med =traj_with_trips_medium.groupby(['SegmentId','StartDate_dmy','StartTime_h']).size().reset_index(name='counts')
    #trips_summary_heavy =traj_with_trips_medium.groupby(['SegmentId','StartDate_dmy','StartTime_h']).size().reset_index(name='counts')
    #trips_summary = pd.DataFrame(traj_df['SegmentId'].value_counts())
        
    trips_summary_med.reset_index(inplace = True)
    #trips_summary_heavy.reset_index(inplace = True)
    
    trips_summary_med = trips_summary_med.rename(columns={"SegmentId": "trips", "index": "SegmentId"})
    #trips_summary_heavy = trips_summary_heavy.rename(columns={"SegmentId": "trips", "index": "SegmentId"})
    
    i_trips_med = trips_summary_med[~trips_summary_med['trips'].str.contains('-')]
    i_trips_med = i_trips_med.rename(columns={"trips" : "trips_i"})
    
    #i_trips_heavy = trips_summary_heavy[~trips_summary_heavy['trips'].str.contains('-')]
    #i_trips_heavy = i_trips_heavy.rename(columns={"trips" : "trips_i"})

    j_trips_med = trips_summary_med[trips_summary_med['trips'].str.contains('-')]
    j_trips_med['SegmentId'] = j_trips_med['trips'].str[1:]
    j_trips_med = j_trips_med.rename(columns={"trips" : "trips_j"})
    
    #j_trips_heavy = trips_summary_heavy[trips_summary_heavy['trips'].str.contains('-')]
    #j_trips_heavy['SegmentId'] = j_trips_heavy['trips'].str[1:]
    #j_trips_heavy = j_trips_heavy.rename(columns={"trips" : "trips_j"})
    
    i_trips_full_med = pd.concat([i_trips_full_med,i_trips_med])
    j_trips_full_med = pd.concat([j_trips_full_med,j_trips_med])
    
    #i_trips_full_heavy = pd.concat([i_trips_full_heavy,i_trips_heavy])
    #j_trips_full_heavy = pd.concat([j_trips_full_heavy,j_trips_heavy])


  0%|          | 0/71 [00:00<?, ?it/s]

\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trajs\trajs.csv.gz


C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  1%|▏         | 1/71 [04:03<4:44:14, 243.63s/it]C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trajs\trajs.csv1.gz


C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  3%|▎         | 2/71 [08:12<4:43:42, 246.70s/it]C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 97%|█████████▋| 69/71 [5:01:00<08:04, 242.44s/it]C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trajs\trajs.csv8.gz


C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 99%|█████████▊| 70/71 [5:06:24<04:26, 266.70s/it]C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trajs\trajs.csv9.gz


C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 71/71 [5:11:58<00:00, 263.64s/it]


In [7]:
i_trips_full_med

,SegmentId,trips_i,StartDate_dmy,StartTime_h,counts
641137,641137,100100179_0,02052019,12,1
641138,641138,100100179_2,02052019,12,1
641139,641139,100100179_3,02052019,12,1
641140,641140,100100179_5,03052019,21,1
641141,641141,100100179_6,16052019,12,1
...,...,...,...,...,...
1943817,1943817,99968971_1,15052019,12,1
1943818,1943818,99979708_0,08052019,04,1
1943819,1943819,99979708_0,27052019,16,1
1943820,1943820,99979708_3,07052019,06,1


In [16]:
j_trips_full_med.head(10)

,SegmentId,trips_j,StartDate_dmy,StartTime_h,counts
0,100026065_0,-100026065_0,14052019,16,1
1,100095907_2,-100095907_2,23052019,20,1
2,100095907_4,-100095907_4,28052019,22,1
3,100163147_0,-100163147_0,03052019,12,1
4,100163147_0,-100163147_0,06052019,08,1
5,100163147_0,-100163147_0,08052019,07,1
6,100163147_0,-100163147_0,15052019,13,1
7,100163147_0,-100163147_0,17052019,07,1
8,100163147_0,-100163147_0,22052019,09,1
9,100163147_0,-100163147_0,28052019,14,1


In [17]:
#trips by an hour
i_trips_upd_hour = i_trips_full_med.groupby(['trips_i','StartDate_dmy','StartTime_h'])['counts'].sum().reset_index(name='sum')
j_trips_upd_hour = j_trips_full_med.groupby(['SegmentId','StartDate_dmy','StartTime_h'])['counts'].sum().reset_index(name='sum')

all_segments_hour = i_trips_upd_hour.merge(j_trips_upd_hour, left_on= 'trips_i', right_on = 'SegmentId', how ='outer')

all_segments_hour = all_segments_hour.fillna(0)
all_segments_hour['total'] = all_segments_hour['sum_x'] + all_segments_hour['sum_y']
all_segments_hour.sort_values(by=['total'])

MemoryError: Unable to allocate 34.4 GiB for an array with shape (4614605549,) and data type int64

In [ ]:
all_segments_hour.to_csv('all_segments_hour.csv')

In [35]:
#trips by a segment
i_trips_upd = i_trips_full_med.groupby('SegmentId')['trips_i'].sum().reset_index(name='sum').sort_values(by=['sum'])
j_trips_upd = i_trips_full_med.groupby('SegmentId')['trips_j'].sum().reset_index(name='sum').sort_values(by=['sum'])

In [13]:
i_trips_upd_hour

,trips_i,StartDate_dmy,StartTime_h,sum
0,100026065_0,02052019,19,1
1,100026065_0,03052019,09,3
2,100026065_0,08052019,14,1
3,100026065_0,09052019,11,1
4,100026065_0,10052019,07,1
...,...,...,...,...
29876554,99980072_0,21052019,15,1
29876555,99980072_0,21052019,21,1
29876556,99980072_0,23052019,02,1
29876557,99980072_0,24052019,19,1


In [38]:
all_segments = i_trips_upd.merge(j_trips_upd, on = 'SegmentId', how ='outer')
all_segments = all_segments.fillna(0)
all_segments['total'] = all_segments['sum_x'] + all_segments['sum_y']
all_segments

,SegmentId,sum_x,sum_y,total
0,5390687_16,1.0,0.0,1.0
1,288575150_0,1.0,0.0,1.0
2,288575152_0,1.0,0.0,1.0
3,288575158_0,1.0,10.0,11.0
4,288575158_1,1.0,0.0,1.0
...,...,...,...,...
684402,703027168_0,0.0,476.0,476.0
684403,703027169_0,0.0,476.0,476.0
684404,853730428_2,0.0,655.0,655.0
684405,174667357_0,0.0,981.0,981.0


In [43]:
all_segments.max()

SegmentId    99980072_0
sum_x             53264
sum_y             12389
total             53264
dtype: object

Comparing INRIX segments with OSM segments

In [90]:
traj_list = i_trips['SegmentId'].tolist()
osm_list = osm['seg_id'].tolist()

In [94]:
set(traj_list) - set(osm_list)

{'519180986_0',
 '5259126_15',
 '430645240_3',
 '340558916_4',
 '371297755_0',
 '8163697_0',
 '52974767_9',
 '5905482_10',
 '5846044_17',
 '23690114_18',
 '487167145_0',
 '5401676_1',
 '197954497_4',
 '147580770_3',
 '12230709_1',
 '739721236_2',
 '129593268_6',
 '29376924_2',
 '90163554_0',
 '779462463_0',
 '65230017_0',
 '746884061_1',
 '5328259_0',
 '188367410_0',
 '69202713_2',
 '94854635_8',
 '285346745_44',
 '24220702_0',
 '215656968_0',
 '121875859_1',
 '107627287_0',
 '34926117_0',
 '35306732_0',
 '6070423_4',
 '395028958_0',
 '6215989_15',
 '245221243_5',
 '169411760_3',
 '12238974_6',
 '6479184_2',
 '645169122_0',
 '611865255_0',
 '280419688_1',
 '418685630_0',
 '339850285_3',
 '12237655_11',
 '65229965_1',
 '494326906_0',
 '487167144_0',
 '31827237_0',
 '5502758_0',
 '421949596_3',
 '336858017_0',
 '6118158_11',
 '141273347_0',
 '31877211_0',
 '107344684_2',
 '6180238_1',
 '224740204_39',
 '241168641_0',
 '34161035_0',
 '367550642_0',
 '33420554_19',
 '6189370_2',
 '13213314

In [92]:
(len(set(traj_list) - set(osm_list))/len(traj_list))*100

20.28307502369982

In [3]:

path =r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trajs' 
file_List= glob.glob(path + "/*.gz")

df = pd.concat(map(pd.read_csv, glob.glob(path + "/*csv.gz")))

C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [48]:
data_path = r'X:\Trans\Freight\Data\OSM'
os.chdir(data_path)
os.getcwd()

'X:\\Trans\\Freight\\Data\\OSM'

In [72]:
os.getcwd()

'X:\\Trans\\Freight\\Data\\OSM'

In [49]:
#reading in OSM files
osm = gpd.read_file("New File Geodatabase.gdb", driver='FileGDB', layer='inrix_osm_region')

In [54]:
sum(all_segments["SegmentId"].duplicated())

0

In [66]:
all_segments.describe()

,sum_x,sum_y,total
count,684407.000000,684407.000000,684407.000000
mean,211.913705,90.929285,302.842990
std,1349.663034,374.043282,1486.500368
min,0.000000,0.000000,1.000000
25%,1.000000,1.000000,3.000000
50%,6.000000,3.000000,11.000000
75%,36.000000,22.000000,67.000000
max,53264.000000,12389.000000,53264.000000


In [58]:
#joining OSM file with Traj segments on SegmentId

OSM_traj = pd.merge(all_segments, osm, left_on='SegmentId', right_on='seg_id')

In [67]:
OSM_traj_limit = OSM_traj[OSM_traj['total']>10]

In [69]:
OSM_traj_limit = gpd.GeoDataFrame(OSM_traj_limit, geometry = OSM_traj_limit.geometry)
OSM_traj_limit.to_file("J:\Projects\INRIX\data\INRIX traj aggregated\OSM_traj_limit.shp")

In [64]:
OSM_traj = gpd.GeoDataFrame(OSM_traj, geometry = OSM_traj.geometry)

In [65]:
OSM_traj.to_file("J:\Projects\INRIX\data\INRIX traj aggregated\OSM_traj.shp")

In [95]:
all_osm1 = gpd.read_file("SegmentReference.shp")

In [96]:
all_osm2 = gpd.read_file("SegmentReference1.shp")

In [71]:
OSM_traj_limit

,SegmentId,sum_x,sum_y,total,seg_id,roadname,length_m,sseg_id,from_node,to_node,compseg_id,frc,tz,heading,complement,Shape_Length,geometry
2,288575158_0,1.0,10.0,11.0,288575158_0,,4.474,288575158,1327013350,2921246169,-288575158_0,6,America/Los_Angeles,Some(0),,14.675721,"MULTILINESTRING ((1195722.025 272817.947, 1195..."
123,6523353_2,1.0,12.0,13.0,6523353_2,Northeast 140th Place,84.700,6523353,53194736,53238573,-6523353_2,5,America/Los_Angeles,Some(74),,278.630743,"MULTILINESTRING ((1308900.187 267925.070, 1309..."
188,287428990_0,1.0,10.0,11.0,287428990_0,,39.527,287428990,2910306417,2910306428,-287428990_0,6,America/Los_Angeles,Some(313),,129.890337,"MULTILINESTRING ((1220621.424 299821.210, 1220..."
446,290560247_0,1.0,11.0,12.0,290560247_0,,83.960,290560247,2940762916,2940762920,-290560247_0,6,America/Los_Angeles,Some(16),,275.622417,"MULTILINESTRING ((1294152.210 172501.528, 1294..."
447,290560247_1,1.0,11.0,12.0,290560247_1,,105.848,290560247,2940762920,2940762912,-290560247_1,6,America/Los_Angeles,Some(346),,347.421940,"MULTILINESTRING ((1294217.007 172712.094, 1294..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458239,646763245_0,0.0,269.0,269.0,646763245_0,,54.721,646763245,6075712775,6075712779,-646763245_0,6,America/Los_Angeles,Some(89),,180.071682,"MULTILINESTRING ((1260453.588 39207.615, 12605..."
458240,733910242_2,0.0,272.0,272.0,733910242_2,,31.173,733910242,6873392923,6873392928,-733910242_2,6,America/Los_Angeles,Some(209),,102.332705,"MULTILINESTRING ((1273014.460 203181.907, 1272..."
458241,54108034_0,0.0,292.0,292.0,54108034_0,Orillia Road South,17.972,54108034,1822189720,1822189671,-54108034_0,5,America/Los_Angeles,Some(206),,58.990100,"MULTILINESTRING ((1284975.058 161266.819, 1284..."
458242,174667357_0,0.0,981.0,981.0,174667357_0,Southeast Homestead Valley Road,98.017,174667357,53022351,53177090,-174667357_0,4,America/Los_Angeles,Some(185),,321.563526,"MULTILINESTRING ((1433838.579 162961.048, 1433..."


In [97]:
osm_list_all1 = all_osm1['seg_id'].tolist()
osm_list_all2 = all_osm2['seg_id'].tolist()

In [101]:
osm_list_all = osm_list_all1+osm_list_all2

In [105]:
len(set(traj_list) - set(osm_list_all))

16738

In [106]:
set(traj_list) - set(osm_list_all)

{'29170674_0',
 '835364313_3',
 '828702402_0',
 '52974767_9',
 '838032370_0',
 '44722133_5',
 '418972075_0',
 '23989837_0',
 '32293246_1',
 '185037548_0',
 '23690114_18',
 '456378049_0',
 '12172101_5',
 '5401676_1',
 '34913770_0',
 '197954497_4',
 '12230709_1',
 '90163554_0',
 '242428541_11',
 '6138496_10',
 '164667208_28',
 '840434483_0',
 '6355724_15',
 '850601991_2',
 '143657596_1',
 '127591105_3',
 '12171203_2',
 '5328259_0',
 '188367410_0',
 '259121540_7',
 '704459997_5',
 '121734200_0',
 '45313453_0',
 '392627018_1',
 '51528346_2',
 '91535847_0',
 '121875859_1',
 '107627287_0',
 '442636581_0',
 '392799886_0',
 '261560693_14',
 '34926117_0',
 '12242128_15',
 '737524650_77',
 '5916867_12',
 '127588925_5',
 '395028958_0',
 '485642300_0',
 '847474027_3',
 '825911635_1',
 '6416923_26',
 '185379375_0',
 '418752036_1',
 '6479856_11',
 '199340032_0',
 '245221243_5',
 '6479184_2',
 '12238974_6',
 '595236871_1',
 '749691051_0',
 '34926932_1',
 '204489576_2',
 '34913442_6',
 '418685630_0',


In [104]:
(len(set(traj_list) - set(osm_list_all)))/len(traj_list)*100

7.380233250292114

In [107]:
del all_osm1
del all_osm2

Check one trip fro one file

In [73]:
path =r'\\AWS-Model10\Model Data 2\INRIX\trip_paths_usa_wa_puget_sound_201905\date=2020-12-29\reportId=45749\v1\data\trajs' 
File_List= glob.glob(path + "/*csv.gz")
traj_df = pd.read_csv(file_, compression='gzip',names = traj_header.columns,sep=',',  error_bad_lines=False)

C:\Users\pbutrina\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [76]:
traj_df[traj_df['TripId'] == 'cc44f2ff32c9c2790d66eac1dda2387c']

,TripId,DeviceId,ProviderId,TripTimezone,TrajIdx,TrajRawDistanceM,TrajRawDurationMillis,SegmentId,SegmentIdx,LengthM,CrossingStartOffsetM,CrossingEndOffsetM,CrossingStartDateUtc,CrossingEndDateUtc,CrossingSpeedKph,OnRoadNetworkSnapCount,ErrorCodes
4,cc44f2ff32c9c2790d66eac1dda2387c,8ef588a5d4e7bb379f3ad9be55aa0c39,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,23119.078003,1720981,-428004181_0,65,4.320,0.0,4.320,2019-05-10T13:53:09.930Z,2019-05-10T13:53:10.272Z,45.530336,0,NaN
675,cc44f2ff32c9c2790d66eac1dda2387c,8ef588a5d4e7bb379f3ad9be55aa0c39,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,23119.078003,1720981,-353714432_0,137,30.917,0.0,30.917,2019-05-10T13:59:04.192Z,2019-05-10T13:59:05.566Z,80.981469,0,NaN
722,cc44f2ff32c9c2790d66eac1dda2387c,8ef588a5d4e7bb379f3ad9be55aa0c39,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,23119.078003,1720981,99094602_0,43,49.937,0.0,49.937,2019-05-10T13:52:32.751Z,2019-05-10T13:52:36.539Z,47.459019,0,NaN
2349,cc44f2ff32c9c2790d66eac1dda2387c,8ef588a5d4e7bb379f3ad9be55aa0c39,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,23119.078003,1720981,-428056967_0,133,44.882,0.0,44.882,2019-05-10T13:58:56.525Z,2019-05-10T13:58:59.510Z,54.118661,1,NaN
3323,cc44f2ff32c9c2790d66eac1dda2387c,8ef588a5d4e7bb379f3ad9be55aa0c39,54229abfcfa5649e7003b83dd4755294,America/Los_Angeles,0,23119.078003,1720981,-428003217_1,47,26.172,0.0,26.172,2019-05-10T13:52:41.747Z,2019-05-10T13:52:43.732Z,47.459019,0,NaN


In [77]:
os.getcwd()

'X:\\Trans\\Freight\\Data\\OSM'